# Fichier pour générer les soumissions et prédictions

### Se sert des fichiers .sav du model et de scaler

In [56]:
# librairies 
#----------------------------------------------
#----- Calculs 
import pandas as pd
import numpy as np
from numpy import mean, std
from math import sqrt, log
#save model 
import pickle 

### Récupération du test set d'évaluation Kaggle

In [57]:
# Lecture du fichier des données tests
#----------------------------------------------
PATH = '.' #modifier si besoin le chemin du fichier

# récupération du fichier contentant l'ensemble des données issues des requêtes
eval_set_kaggle = pd.read_csv(PATH + '//test_set.csv')

# récupération du fichier contenant les features des hotels
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])

eval_set_kaggle_detailed = eval_set_kaggle.join(hotels, on=['hotel_id', 'city'])
idx = eval_set_kaggle_detailed ["index"]
del eval_set_kaggle_detailed ["index"]
eval_set_kaggle_detailed.head()
eval_set_kaggle_detailed.rename(columns = {'order_requests':"request_number"}, inplace = True)
eval_set_kaggle_detailed.reindex(columns=['hotel_id','stock','city','date','language','mobile', "avatar_id","request_number" ,"group","brand","parking", "pool","children_policy"])

,hotel_id,stock,city,date,language,mobile,avatar_id,request_number,group,brand,parking,pool,children_policy
0,161,46,vilnius,21,romanian,0,1,1,Boss Western,J.Halliday Inn,1,0,0
1,187,32,vilnius,21,romanian,0,1,1,Accar Hotels,Marcure,1,1,0
2,279,12,vilnius,21,romanian,0,1,1,Independant,Independant,1,0,0
3,395,10,vilnius,21,romanian,0,1,1,Accar Hotels,Ibas,0,0,0
4,488,42,vilnius,21,romanian,0,1,1,Accar Hotels,Safitel,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,987,1,rome,5,irish,0,794,843,Accar Hotels,Ibas,1,0,0
6640,26,1,vienna,1,irish,1,794,844,Accar Hotels,Marcure,1,0,0
6641,263,0,vienna,1,irish,1,794,844,Boss Western,Boss Western,1,0,0
6642,456,0,vienna,1,irish,1,794,844,Yin Yang,Ardisson,1,0,0


In [58]:
# Assigner le bon type aux variables
#----------------------------------------------
#--- Convert to integer: 
int_list = ["date","avatar_id","hotel_id","stock","request_number"] 
eval_set_kaggle_detailed[int_list] = eval_set_kaggle_detailed[int_list].astype(int) 
#--- Convert to categorical: 
eval_set_kaggle_detailed["city"] = pd.Categorical(eval_set_kaggle_detailed["city"],ordered=False)
eval_set_kaggle_detailed["language"] = pd.Categorical(eval_set_kaggle_detailed["language"],ordered=False)
eval_set_kaggle_detailed["mobile"] = pd.Categorical(eval_set_kaggle_detailed["mobile"],ordered=False)
eval_set_kaggle_detailed["parking"] = pd.Categorical(eval_set_kaggle_detailed["parking"],ordered=False)
eval_set_kaggle_detailed["pool"] = pd.Categorical(eval_set_kaggle_detailed["pool"],ordered=False)
eval_set_kaggle_detailed["children_policy"] = pd.Categorical(eval_set_kaggle_detailed["children_policy"],ordered=False)
eval_set_kaggle_detailed["group"] = pd.Categorical(eval_set_kaggle_detailed["group"],ordered=False)
eval_set_kaggle_detailed["brand"] = pd.Categorical(eval_set_kaggle_detailed["brand"],ordered=False)

### Tranformation de la variable quantitative 'stock' comme pour l'apprentissage

In [59]:
eval_set_kaggle_detailed["stock"]=eval_set_kaggle_detailed["stock"].map(lambda x: log(x+1))

In [60]:
# Variables explicatives
#-----------------------------------------------
# Liste des variables categorical: 
cat_list1 = ["city", "language", "brand","mobile","parking","pool","children_policy"] #avec brand
cat_list2 = ["city", "language", "group","mobile","parking","pool","children_policy"] #avec group
cat_list3 = ["city", "language", "brand", "group","mobile","parking","pool","children_policy"] #avec brand & group

# Liste des variables quantitatives: 
quant_list1 = ["hotel_id", "stock","request_number", "date"] #on a enlevé avatar_id et price (=variable à expliquer)

# Création de la dataframe train
eval_set_kaggle_detailed_pricingDum  = pd.get_dummies(eval_set_kaggle_detailed[cat_list2],drop_first = True)
eval_set_kaggle_detailed_pricingQuant = eval_set_kaggle_detailed[quant_list1]

dfC_eval_set_kaggle = pd.concat([eval_set_kaggle_detailed_pricingDum ,eval_set_kaggle_detailed_pricingQuant],axis=1)
dfC_eval_set_kaggle.columns

Index(['city_copenhagen', 'city_madrid', 'city_paris', 'city_rome',
       'city_sofia', 'city_valletta', 'city_vienna', 'city_vilnius',
       'language_belgian', 'language_bulgarian', 'language_croatian',
       'language_cypriot', 'language_czech', 'language_danish',
       'language_dutch', 'language_estonian', 'language_finnish',
       'language_french', 'language_german', 'language_greek',
       'language_hungarian', 'language_irish', 'language_italian',
       'language_latvian', 'language_lithuanian', 'language_luxembourgish',
       'language_maltese', 'language_polish', 'language_portuguese',
       'language_romanian', 'language_slovakian', 'language_slovene',
       'language_spanish', 'language_swedish', 'group_Boss Western',
       'group_Chillton Worldwide', 'group_Independant',
       'group_Morriott International', 'group_Yin Yang', 'mobile_1',
       'parking_1', 'pool_1', 'children_policy_1', 'children_policy_2',
       'hotel_id', 'stock', 'request_number', 'date'

In [61]:
eval_set_kaggle_detailed

,request_number,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
0,1,vilnius,21,romanian,0,1,161,3.850148,Boss Western,J.Halliday Inn,1,0,0
1,1,vilnius,21,romanian,0,1,187,3.496508,Accar Hotels,Marcure,1,1,0
2,1,vilnius,21,romanian,0,1,279,2.564949,Independant,Independant,1,0,0
3,1,vilnius,21,romanian,0,1,395,2.397895,Accar Hotels,Ibas,0,0,0
4,1,vilnius,21,romanian,0,1,488,3.761200,Accar Hotels,Safitel,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,843,rome,5,irish,0,794,987,0.693147,Accar Hotels,Ibas,1,0,0
6640,844,vienna,1,irish,1,794,26,0.693147,Accar Hotels,Marcure,1,0,0
6641,844,vienna,1,irish,1,794,263,0.000000,Boss Western,Boss Western,1,0,0
6642,844,vienna,1,irish,1,794,456,0.000000,Yin Yang,Ardisson,1,0,0


### Scaler tranformation & chargement du modèle

Les noms sont à adapter : charger les modèles et standard scaler correspondant(date)

In [62]:
model_folder = '//models_sav/'
scaler = pickle.load(open(PATH + model_folder + 'Standard_Scaler.sav', 'rb')) #chargement
Xr_eval = scaler.transform(dfC_eval_set_kaggle)
# transformation de Xr_train et Xr_test (numpy array) en dataframe
Xr_eval = pd.DataFrame(Xr_eval, index=dfC_eval_set_kaggle.index, columns=dfC_eval_set_kaggle.columns)

In [63]:
#filename = 'X_gboost_tuned_model.sav' # variable
filename = 'RDF_model.sav'
loaded_model = pickle.load(open(PATH + model_folder + filename, 'rb'))

### Prédiction et enregistrement csv

In [64]:
pred_eval = loaded_model.predict(Xr_eval)
predict =pd.DataFrame(list(zip(idx, pred_eval**2)), columns=['index', 'price']) #### Mettre la prediction au carré car tranfor sqrt dans la transformation des données
predict.to_csv('submission.csv', index=False)
print(predict)

      index       price
0         0  109.297633
1         1  139.264517
2         2   54.729440
3         3   85.573369
4         4  140.295509
...     ...         ...
6639   6639  159.471156
6640   6640  187.310390
6641   6641  134.362108
6642   6642  162.081287
6643   6643  155.765055

[6644 rows x 2 columns]


In [65]:
loaded_model

RandomForestRegressor(n_estimators=500, oob_score=True)